### Setup Imports

In [1]:
import os
import sys
import duckdb
import pandas as pd
from huggingface_hub import hf_hub_download

In [2]:
# Add project source code to path for grabbing functions
sys.path.append("../src")

#### Import all security rules and llm context helper

In [ ]:
from security_rules_build import combined_rules, build_llm_context

In [ ]:
#connect to the database
con = duckdb.connect("../data/logs.duckdb")
#sample check
row_count = con.execute("SELECT COUNT(*) FROM logs").fetchone()[0]
print("Connected, total rows:", row_count)

Connected, total rows: 1920624


In [5]:
# Laying out the dictionary keys which are the names of each security rule dataframe. 
results = run_all_rules(con)
results.keys()

dict_keys(['failed_logins', 'failed_by_ip', 'brute_force', 'priv_esc', 'account_lockout', 'log_cleared', 'susp_linux', 'susp_win', 'rdp_logins', 'ssh_root_logins', 'failed_count_windows'])

In [6]:
context_summary = build_llm_context(results, examples=2)
print(context_summary)  # preview only first 1500 characters

🧩 Context summary ready for LLM.

=== Security Detection Summary ===

🔹 FAILED_LOGINS (Windows: 63, Ubuntu: 16597)
Total: 16660 events
  AGENT_NAME                                                                                      Content          src_ip        dst_ip           entity_id    City Country                date day hour minute second           timestamp
UBUNTU_AGENT sshd[2756135]: Failed password for invalid user site from 185.213.165.107 port 48410 ssh2... 185.213.165.107 172.21.35.175 Nisir@172.21.35.175  Tehran    Iran 2025-05-15 09:25:56   3    9     25     56 2025-05-15 09:25:56
UBUNTU_AGENT                 sshd[2653867]: Failed password for root from 218.92.0.177 port 40367 ssh2...    218.92.0.177 172.21.35.175 Nisir@172.21.35.175 Unknown   China 2025-05-14 10:14:47   2   10     14     47 2025-05-14 10:14:47

🔹 FAILED_BY_IP
Total: 9 events
      src_ip attempts
172.20.82.21       55
 172.20.0.39       33

🔹 BRUTE_FORCE
Total: 12 events
      src_ip                  

### Setup and keep local LLM ready 

In [7]:
# storing the model path into a variable
model_path = hf_hub_download(
    repo_id="TheBloke/Llama-2-13B-chat-GGUF",
    filename="llama-2-13b-chat.Q5_K_M.gguf"
)

In [8]:
# Load the LLM
from llama_cpp import Llama, llama_print_system_info

llm = Llama(
    model_path= model_path,
    n_ctx=4096,
    n_threads=12,
    n_gpu_layers=42,
    n_batch=512
)

#llama_print_system_info()
print("LLaMA 13B model loaded!")

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4070, compute capability 8.9, VMM: yes
llama_model_load_from_file_impl: using device CUDA0 (NVIDIA GeForce RTX 4070) - 11094 MiB free
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from C:\Users\Ishaq\.cache\huggingface\hub\models--TheBloke--Llama-2-13B-chat-GGUF\snapshots\4458acc949de0a9914c3eab623904d4fe999050a\llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.emb

LLaMA 13B model loaded!


CUDA : ARCHS = 890 | USE_GRAPHS = 1 | PEER_MAX_BATCH_SIZE = 128 | CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | LLAMAFILE = 1 | OPENMP = 1 | AARCH64_REPACK = 1 | 
Model metadata: {'general.name': 'LLaMA v2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'llama.rope.dimension_count': '128', 'llama.embedding_length': '5120', 'llama.block_count': '40', 'llama.feed_forward_length': '13824', 'llama.attention.head_count': '40', 'tokenizer.ggml.eos_token_id': '2', 'general.file_type': '17', 'llama.attention.head_count_kv': '40', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'tokenizer.ggml.model': 'llama', 'general.quantization_version': '2', 'tokenizer.ggml.bos_token_id': '1', 'tokenizer.ggml.unknown_token_id': '0'}
Using fallback chat format: llama-2


In [41]:
print(llm.metadata)

{'general.name': 'LLaMA v2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'llama.rope.dimension_count': '128', 'llama.embedding_length': '5120', 'llama.block_count': '40', 'llama.feed_forward_length': '13824', 'llama.attention.head_count': '40', 'tokenizer.ggml.eos_token_id': '2', 'general.file_type': '17', 'llama.attention.head_count_kv': '40', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'tokenizer.ggml.model': 'llama', 'general.quantization_version': '2', 'tokenizer.ggml.bos_token_id': '1', 'tokenizer.ggml.unknown_token_id': '0'}


#### First test using LLM context helper

In [45]:
prompt =f"""
You are a SOC (Security Operations Center) analyst.
Analyze the following structured SIEM detections.

For each rule, describe:
1. What the detected activity means,
2. The potential threat or attack pattern it indicates,
3. Recommended actions oar mitigations.

Here are the detections:
{context_summary[:7000]}
""" 

In [46]:
response = llm.create_chat_completion(
    messages=[
        {"role": "system", "content": "You are an expert SOC() analyst."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=700,
    temperature=0.4,
)

print("LLaMA Analysis:")
print(response["choices"][0]["message"]["content"])

llama_perf_context_print:        load time =   17187.48 ms
llama_perf_context_print: prompt eval time =   17184.73 ms /  2363 tokens (    7.27 ms per token,   137.51 tokens per second)
llama_perf_context_print:        eval time =   77145.54 ms /   699 runs   (  110.37 ms per token,     9.06 tokens per second)
llama_perf_context_print:       total time =   94890.24 ms /  3062 tokens


LLaMA Analysis:
  As a SOC analyst, I have analyzed the provided SIEM detections and recommend the following actions and mitigations based on the detected activity:

1. 🔹 Rule: FAILED_LOGINS (Windows: 63, Ubuntu: 16597)
Total records: 16660

Potential threat or attack pattern: Brute-force attacks or password guessing attempts.

Recommended actions and mitigations:

a. Implement strong password policies and enforce password complexity requirements.

b. Enable multi-factor authentication (MFA) to add an extra layer of security.

c. Restrict login attempts to prevent excessive attempts from the same IP address.

d. Monitor and review failed login attempts regularly to detect and respond to potential attacks.

2. 🔹 Rule: FAILED_BY_IP
Total records: 9

Potential threat or attack pattern: Brute-force attacks or password guessing attempts from specific IP addresses.

Recommended actions and mitigations:

a. Block or restrict access from the identified IP addresses.

b. Implement IP address bl

### Adding raw data with context data for an output. 

In [9]:
raw_sample = con.execute("SELECT * FROM logs LIMIT 50").fetchdf()
raw_sample["Content"] = raw_sample["Content"].astype(str).str.slice(0, 200) + "..."
raw_text = raw_sample.to_string(index=False)

In [12]:
''''context_summary = build_llm_context(results, examples=3)

context_full = f"""
=== STRUCTURED SECURITY SUMMARY ===
{context_summary[:5000]}

=== RAW LOG SAMPLE ===
{raw_text}

Analyze both the structured detections and the raw log data together.
Identify major threats, link raw evidence to detections, and describe what an SOC analyst should do next.
"""
'''

context_summary = build_llm_context(results, examples=3)

context_full = f"""
=== RAW LOG SAMPLE ===
{raw_text}

Analyze the  the raw log data.
Identify major threats, link raw evidence to detections, and describe what an SOC analyst should do next.
"""

In [13]:
response = llm.create_chat_completion(
    messages=[
        {"role": "system", "content": "You are a senior SOC analyst reviewing SIEM data."},
        {"role": "user", "content": context_full}
    ],
    max_tokens=700,
    temperature=0.5,
)

print("🧠 Full Analysis:\n")
print(response["choices"][0]["message"]["content"])

ValueError: Requested tokens (8744) exceed context window of 4096

In [57]:
prompt =f"""
You are a SOC (Security Operations Center) analyst.
Analyze the following structured SIEM detections.

For each rule, describe:
1. What the detected activity means,
2. The potential threat or attack pattern it indicates,
3. Recommended actions oar mitigations.

Here are the detections:
{results}"""

In [55]:
response = llm.create_chat_completion(
    messages=[
        {"role": "system", "content": "You are a senior SOC analyst reviewing SIEM data."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=700,
    temperature=0.5,
)

print("Full Analysis:\n")
print(response["choices"][0]["message"]["content"])

ValueError: Requested tokens (17091) exceed context window of 4096